# Construction Bid Comparison - Feasibility Test

This notebook tests semantic matching capabilities for construction bid comparisons.

## Setup Instructions:
1. Upload this notebook to Google Colab
2. Add your API key to Colab Secrets (key icon on left) with name 'GOOGLE_API_KEY'
3. Upload your 3 test files to the session storage
4. Run all cells

## What This Tests:
- Semantic matching across different description formats
- Comparison table generation
- CSV export capability

In [ ]:
# Install dependencies
!pip install -q google-generativeai pandas

In [ ]:
# Import libraries
import google.generativeai as genai
from google.colab import userdata, files
import pandas as pd
import json

# Configure API
try:
    API_KEY = userdata.get('GOOGLE_API_KEY')
except:
    API_KEY = input('Enter your Gemini API key: ')

genai.configure(api_key=API_KEY)
print('✓ Gemini configured successfully')

In [ ]:
# Upload test files
print('Upload your 3 CSV/Excel files:')
print('1. Base file (presupuesto base)')
print('2. Vendor 1 comparison file')
print('3. Vendor 2 comparison file')
print()
uploaded = files.upload()

file_names = list(uploaded.keys())
print(f'\n✓ Uploaded {len(file_names)} files: {file_names}')

In [ ]:
# Load CSV data
def load_file_data(filename):
    if filename.endswith('.csv'):
        with open(filename, 'r', encoding='utf-8') as f:
            return f.read()
    elif filename.endswith(('.xlsx', '.xls')):
        df = pd.read_excel(filename)
        return df.to_csv(index=False)
    else:
        raise ValueError(f'Unsupported file type: {filename}')

# Load all files
file_data = {}
for fname in file_names:
    file_data[fname] = load_file_data(fname)
    print(f'✓ Loaded {fname}')

print(f'\nTotal files loaded: {len(file_data)}')

In [ ]:
# Specify which file is the base
print('Available files:')
for i, fname in enumerate(file_names):
    print(f'{i+1}. {fname}')

base_idx = int(input('\nEnter number of BASE file: ')) - 1
base_file = file_names[base_idx]
comparison_files = [f for f in file_names if f != base_file]

print(f'\n✓ Base file: {base_file}')
print(f'✓ Comparison files: {comparison_files}')

## Test 1: Semantic Matching

This test asks Gemini to identify matching items across files using semantic understanding.

In [ ]:
# Create prompt for semantic matching
prompt = f"""
I need help comparing construction bid items across multiple files. The items are described 
differently but many mean the same thing. Use SEMANTIC matching to identify equivalent items.

BASE FILE ({base_file}):
{file_data[base_file]}

"""

for i, comp_file in enumerate(comparison_files, 1):
    prompt += f"""
COMPARISON FILE {i} ({comp_file}):
{file_data[comp_file]}

"""

prompt += """
TASK:
For each CONCRETE item in the BASE file (items with actual prices, NOT section headers),
identify matching items from the comparison files.

IMPORTANT:
- Use SEMANTIC matching (not exact text matching)
- Consider synonyms, abbreviations, word order variations
- "LOCALIZACIÓN Y REPLANTEO POR METRO CUADRADO..." should match "LOCALIZACIÓN Y REPLANTEO"
- "DEMOLICIÓN DE MUROS" should match "DEMOLER MURO"

For EACH base item with a price, provide:
1. Base item number and description (abbreviated to 40 chars)
2. Matches from each comparison file (item number + description, or "NOT FOUND")
3. Match confidence (HIGH/MEDIUM/LOW) for each match
4. Brief reasoning

Be thorough and systematic.
"""

print('Sending request to Gemini...')
print('(This may take 20-40 seconds...)\n')

In [ ]:
# Send to Gemini
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content(prompt)

print('='*60)
print('SEMANTIC MATCHING RESULTS')
print('='*60)
print()
print(response.text)
print()
print('='*60)

## Test 2: Generate Comparison Table

Now ask Gemini to format the results as a structured comparison table.

In [ ]:
# Generate comparison table
table_prompt = f"""
Based on your analysis above, create a comparison table in markdown format.

Columns:
- Base Item (abbreviated)
- Base Unit
- Base Price
"""

for i, comp_file in enumerate(comparison_files, 1):
    table_prompt += f"""
- {comp_file} Item
- {comp_file} Unit
- {comp_file} Price
- Match (✓/✗)
"""

table_prompt += """

Rules:
- Include only concrete items with prices (not section headers)
- Use abbreviated descriptions (max 30 chars)
- Show actual prices from the data
- Use ✓ for matches, ✗ for not found
- At the end, show match statistics

Make it accurate and complete.
"""

response = model.generate_content(table_prompt)

print('='*60)
print('COMPARISON TABLE')
print('='*60)
print()
print(response.text)
print()
print('='*60)

## Test 3: Export as CSV

Get the comparison data in CSV format for Excel import.

In [ ]:
# Generate CSV format
csv_headers = ['Base_Item', 'Base_Unit', 'Base_Price']
for i, comp_file in enumerate(comparison_files, 1):
    csv_headers.extend([f'File{i}_Item', f'File{i}_Unit', f'File{i}_Price', f'File{i}_Match'])

csv_prompt = f"""
Now generate the comparison data in CSV format (comma-separated values).

CSV Headers:
{','.join(csv_headers)}

Rules:
- Short item descriptions (max 30 chars)
- Numeric prices only (remove $ symbols and commas)
- YES/NO for match columns
- One row per base item
- Enclose text fields in quotes if they contain commas

Provide ONLY the CSV data (no explanations, no markdown code blocks).
Start directly with the header row.
"""

response = model.generate_content(csv_prompt)

print('='*60)
print('CSV OUTPUT')
print('='*60)
print()
print(response.text)
print()
print('='*60)

# Save to file
with open('comparison_results.csv', 'w', encoding='utf-8') as f:
    # Remove markdown code blocks if present
    csv_data = response.text.replace('```csv', '').replace('```', '').strip()
    f.write(csv_data)

print('\n✓ Saved to comparison_results.csv')
print('\nDownload the file:')
files.download('comparison_results.csv')

## Evaluation

Review the results above to evaluate:

### Accuracy Check:
- Did Gemini correctly identify semantic matches?
- Were synonyms properly matched? (e.g., DEMOLICIÓN ↔ DEMOLER)
- Were word order variations handled? (e.g., LOCALIZACIÓN Y REPLANTEO ↔ REPLANTEO Y LOCALIZACIÓN)
- Were abbreviations expanded correctly?

### Expected Performance:
- **High accuracy (85-95%)**: Most matches correct
- **Medium accuracy (70-85%)**: Some errors, needs review
- **Low accuracy (<70%)**: Approach needs refinement

### Next Steps:
If accuracy is good (>80%), proceed with full implementation!

## Feasibility Conclusion:

✅ **If matches look good**: The semantic matching approach WORKS for your use case

✅ **Next**: Implement full solution with:
- Structured table extraction (pdfplumber + pandas)
- Hybrid matching (embeddings + LLM)
- Excel export with formatting
- Manual review workflow

📊 **Implementation**: 1-2 weeks for production-ready solution